In [3]:
# import torch
# from torch.optim import Adam
# import torch.nn as nn
# from torch.autograd import Variable
# import torch.nn.functional as F
from torch.nn import Embedding

# import pandas as pd
# import numpy as np

# import matplotlib.pyplot as plt
# import seaborn as sns

from typing import Tuple, List, TypeVar, Iterable
from typing_extensions import TypeAlias
from enum import Enum, auto
from math import ceil
from torch import BoolTensor, LongTensor, sparse_coo_tensor, ones, tensor
from functools import reduce
from typing import cast, Callable
import numpy as np
from itertools import chain

class Label(Enum):
  touhou = 0
  hololive = auto()
  marisa = auto()
  reimu = auto()
  youmu = auto()
  sakuya = auto()
  flandre = auto()
  reiuji = auto()
  reisen = auto()
  tewi = auto()
  patchouli = auto()
  aya = auto()
  pekora = auto()
  kronii = auto()
  gura = auto()
  suisei = auto()
  ame = auto()
  noel = auto()
  subaru = auto()
  kiara = auto()
  black_hair = auto()
  silver_hair = auto()
  blue_hair = auto()
  blonde_hair = auto()
  purple_hair = auto()
  orange_hair = auto()
  bunny_ears = auto()
  bird_person = auto()

# def caption_to_tensor(caption: Tuple[Label, ...]) -> BoolTensor:
#   sparse_coo_tensor(
#     indices=tensor(caption, dtype=bool),
#     values=ones(len(caption), dtype=bool),
#     dtype=bool)

T = TypeVar('T')
# _CountExtractor: TypeAlias = Callable[[T], int]
# _Counter: TypeAlias = Callable[[int, T], int]
_Caption: TypeAlias = Tuple[Label, ...]
_Captions: TypeAlias = Tuple[_Caption, ...]
# _FlattenedCaptions: TypeAlias = _Caption

# def make_counter(extract_count: _CountExtractor[T]) -> _Counter[T]:
#   def count(acc: int, next: T) -> int:
#     return acc + extract_count(next)
#   return count

# def extract_count(labels: _Caption) -> int:
#   return len(labels)

def make_row_indices(enumerated: Tuple[int, _Caption]) -> Tuple[int, ...]:
  (ix, labels) = enumerated
  return (ix,) * len(labels)

def flatten(captions: Iterable[Tuple[T, ...]]) -> Iterable[T]:
  return chain.from_iterable(captions)

def get_value(label: Label) -> int:
  return label.value

def captions_to_tensor(captions: _Captions) -> BoolTensor:
  # counter: _Counter[Label] = make_counter(extract_count)
  # element_count: int = reduce(counter, captions, 0)
  row_indices: Tuple[int, ...] = tuple(flatten(map(make_row_indices, enumerate(captions))))
  labels: Tuple[int, ...] = tuple(map(get_value, flatten(captions)))

  indices_nominal: Tuple[Tuple[int, ...], Tuple[int, ...]] = (row_indices, labels)
  return sparse_coo_tensor(
    indices=LongTensor(indices_nominal),
    values=ones(len(row_indices), dtype=bool),
    dtype=bool)

captions: _Captions = (
  (Label.touhou, Label.marisa, Label.blonde_hair),
  (Label.touhou, Label.reimu, Label.black_hair),
  (Label.touhou, Label.youmu, Label.silver_hair),
  (Label.touhou, Label.sakuya, Label.silver_hair),
  (Label.touhou, Label.flandre, Label.blonde_hair),
  (Label.touhou, Label.reiuji, Label.black_hair, Label.bird_person),
  (Label.touhou, Label.reisen, Label.purple_hair, Label.bunny_ears),
  (Label.touhou, Label.tewi, Label.black_hair, Label.bunny_ears),
  (Label.touhou, Label.patchouli, Label.purple_hair),
  (Label.touhou, Label.aya, Label.black_hair, Label.black_hair),
  (Label.hololive, Label.pekora, Label.blue_hair, Label.bunny_ears),
  (Label.hololive, Label.kronii, Label.blue_hair),
  (Label.hololive, Label.suisei, Label.blue_hair),
  (Label.hololive, Label.gura, Label.silver_hair),
  (Label.hololive, Label.noel, Label.silver_hair),
  (Label.hololive, Label.ame, Label.blonde_hair),
  (Label.hololive, Label.subaru, Label.black_hair, Label.bird_person),
  (Label.hololive, Label.kiara, Label.black_hair, Label.bird_person),
)

caption_tensor: BoolTensor = captions_to_tensor(captions[:2])

vocab_size=len(Label)
# t5-small compressed 32100 vocab tokens into 512 dims
# there's plenty of range per bfloat16 to represent a variety of tokens
embedding_dim=ceil(512/32100 * vocab_size)

model = Embedding(num_embeddings=vocab_size,
                  embedding_dim=embedding_dim,
                  sparse=True)

